In [4]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import numpy as np
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt
import tensorflow as tf

img_folder = 'img_align_celeba\\'
imagens_atributes = 'celeb_attributes.csv'
videos_table_path_validation = 'celeb_bbox.csv'
max_cases_per_split = 10000
batch_size = 16
preprocess_training_test = False
preprocess_validation = True
input_size = (128,128)

In [5]:
input_size = (128,128)
images = pd.read_csv(imagens_atributes, sep=";")
bbox = pd.read_csv(videos_table_path_validation, sep=";")
data = images.merge(bbox)
data.loc[data['Male']==-1, 'Male'] = 0

# BALANCE
male = data[data['Male']==1]
female = data[data['Male']!=1].head(len(male))
data = pd.concat([male, female], ignore_index=True)

#SHUFFLE
data = data.sample(frac=1)

def load_reshape_img(fname):
    img = tf.keras.preprocessing.image.load_img(fname)
    x = tf.keras.preprocessing.image.img_to_array(img)/255.
    x = x.reshape((1,) + x.shape)

    return x

def decode_img(img):
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)    
    return img

def process_path(index):
    file_path = img_folder + '\\' + data.iloc[index]['file']
    img = tf.io.read_file(file_path, )
    img = decode_img(img)
    #(x,y,w,h) = data.iloc[index][['x_1','y_1','width', 'height']].values
    #img = img[x:x+w,y:y+h]
    #img = cv.resize(img, input_size)
    return img

def process_dataframe(dataframe):
    with tf.device('/cpu:0'):
        xs = np.array([load_reshape_img(img_folder + fname) for fname in dataframe.file])
        xs = xs.reshape(xs.shape[0], 218, 178, 3)
        ys = tf.keras.utils.to_categorical(dataframe['Male'], 2)
    return xs, ys

train_datagen  = tf.keras.preprocessing.image.ImageDataGenerator(
  #preprocessing_function=preprocess_input,
  rotation_range=30,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True
)

In [6]:
IMG_SHAPE = (218, 178, 3)
base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax'),
    ])

# for layer in model.layers[:249]:#52]:
#     layer.trainable = False

# x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
# x = tf.keras.layers.Dense(1024, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.5)(x)
# x = tf.keras.layers.Dense(512, activation='relu')(x)
# x = tf.keras.layers.Dense(1, activation='sigmoid')(x)


# model = tf.keras.Model(base_model.input, x)
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy(name='accuracy')])#, tf.keras.metrics.Precision(name='precision'),tf.keras.metrics.Recall(name='recall')]) #

In [5]:
model.save('cnn_inceptionv3_pretrainned_gender_recognition_v3')

INFO:tensorflow:Assets written to: cnn_inceptionv3_pretrainned_gender_recognition_v3\assets


In [3]:
model = tf.keras.models.load_model('cnn_inceptionv3_pretrainned_gender_recognition_v3')

In [7]:
count = 0
split_count = int(len(data)/max_cases_per_split)
for split in np.array_split(data, split_count):
    print('Split' + str(count))
    train_count = int(len(split) * 0.7)
    train_df = split.head(train_count)
    test_df = split.tail(len(split) - train_count)
    x_train, y_train = process_dataframe(train_df)
    x_test, y_test = process_dataframe(test_df)
    train_datagen.fit(x_train)
    train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size,)
    
    history = model.fit(train_generator, steps_per_epoch=train_count/batch_size,  epochs=10, callbacks=[ tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)])
    print('####################################################################################')
    print('#############################Split' + str(count) + 'Test ###########################')
    print('####################################################################################')
    test_loss, test_acc = model.evaluate(x=x_test, y=y_test, verbose=0, batch_size=1)
    print('Test accuracy:' + str( test_acc))
    print('####################################################################################')
    
    model.save('cnn_inceptionv3_pretrainned_gender_recognition_v3')
    count += 1

Split0
Epoch 1/10
462/461 [==============================] - 87s 187ms/step - loss: 0.3361 - accuracy: 0.8651
Epoch 2/10
462/461 [==============================] - 80s 173ms/step - loss: 0.2814 - accuracy: 0.8813
Epoch 3/10
145/461 [========>.....................] - ETA: 54s - loss: 0.2708 - accuracy: 0.8905

KeyboardInterrupt: 

In [18]:
test_loss, test_acc = model.evaluate(x=x_test, y=y_test, verbose=0, batch_size=1)
print('Test accuracy:' + str( test_acc))# + 'Test precision:' + str( test_prec) + 'Test recall:' + str( test_rec))

Test accuracy:1.0


In [4]:
split_count = int(len(data)/max_cases_per_split)
split = np.array_split(data, split_count)[0]
train_count = int(len(split) * 0.7)
train_df = split.head(train_count)
test_df = split.tail(len(split) - train_count)
#x_train, y_train = process_dataframe(train_df)
x_test, y_test = process_dataframe(test_df)

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [16]:
preds = [np.argmax(model(np.expand_dims(x,0))) for x in x_test]

In [4]:
features = np.array([list(x[0].numpy()) for x in list(data_ds)])
labels = np.array([x[1].numpy() for x in list(data_ds)])

In [9]:
img = np.expand_dims( features[1], 0)
model.predict(img)

array([[1.]], dtype=float32)

In [15]:
r = [int(x[1]) for x in list(test_ds)]

In [16]:
([x[1].numpy() for x in list(test_ds)])

array([0, 0, 0, ..., 1, 0, 1])

In [5]:
features = np.array([list(x[0].numpy()) for x in list(test_ds)])
labels = np.array(r)
test_loss, test_acc, test_prec, test_rec = model.evaluate(x=features, y=labels, verbose=0)

print('\nTest accuracy:', test_acc)
print('\nTest precision:', test_prec)
print('\nTest recall:', test_rec)

NameError: name 'test_ds' is not defined

In [61]:
def load_caffe_model():
    return cv.dnn.readNetFromCaffe('deploy_gender.prototxt', 'gender_net.caffemodel')

gender_net = load_caffe_model()

In [65]:
preds = []
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
for img in features[:100]:
    blob = cv.dnn.blobFromImage(img, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
    gender_net.setInput(blob)
    gender_preds = gender_net.forward()
    preds.append(gender_preds[0].argmax())

array([[0.01312369, 0.98687637]], dtype=float32)

In [68]:
labels[:10]

array([False,  True, False, False, False, False,  True,  True, False,
        True])